<img title="a title" alt="Alt text" src="social.png" width="100" height="100"> 

# Análise de Repositórios GitHub por Linguagem de Programação

## 🌍 Introdução
Este projeto tem como objetivo criar um dataset rico e estruturado com informações dos repositórios mais relevantes do GitHub, organizados pelas 10 linguagens de programação mais populares em 2025. Com esses dados, podemos responder perguntas como:

- Qual linguagem tem os projetos mais estrelados?
- Há correlação entre o número de contribuidores e a atividade do repositório?
- Quais licenças são mais comuns em projetos open-source?
- Como a localização geográfica dos donos influencia a popularidade dos repositórios?

O dataset gerado pode ser usado para:

✅ Identificação de tendências no desenvolvimento de software

✅ Análise de comunidades open-source

✅ Tomada de decisões para contribuições ou adoção de tecnologias

In [11]:
# !pip install pandas
# !pip install requests

### Imports

In [12]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import random
import os

## 🔧 Configuração de Múltiplos Tokens GitHub
Este bloco configura um sistema de rotação de tokens para evitar limites da API do GitHub. A rotação aleatória distribui as requisições entre diferentes tokens, aumentando significativamente o limite de rate da API.

**Benefícios:**

- ✅ Evita bloqueios por rate limit
- ✅ Permite coleta de grandes volumes de dados
- ✅ Distribui carga entre múltiplas chaves de API
- ✅ Fallback automático em caso de token inválido

In [ ]:
TOKENS = [
    "TOKENS"
]   

def get_headers():
    return {"Authorization": f"token {random.choice(TOKENS)}"}

## 🎯 Configuração de Parâmetros de Extração
🎯 Configuração de Parâmetros de Extração
Define as configurações principais para a coleta sistemática de dados:
**Linguagens Selecionadas**: As 10 linguagens mais populares em 2025
**Estratégia de Paginação**: 15 páginas × 100 repositórios = 1.500 repos por linguagem
**Total Estimado**: ~15.000 repositórios coletados

**Critérios de Seleção:**

- Repositórios ordenados por popularidade (stars)
- Foco em projetos ativos e relevantes
- Cobertura equilibrada entre diferentes linguagens

In [14]:
LANGUAGES = ["Python", "JavaScript", "Java", "C#", "C++", "TypeScript", "Go", "Rust", "Kotlin", "Swift"]
PER_PAGE = 100
PAGES = 15
OUTPUT_FILE = "../dados/github_repos_completos.csv"

# 👤 Enriquecimento de Dados do Proprietário

Esta função acessa o perfil completo do dono de cada repositório para extrair informações contextuais importantes:

**Dados Coletados:**
- **Tipo de Conta:** Usuário individual vs Organização
- **Portfólio:** Quantidade de repositórios públicos
- **Localização:** Informação geográfica declarada

**Tratamento de Erros:**
- Retry automático em falhas temporárias
- Valores padrão para perfis inacessíveis
- Log detalhado de erros para debugging

In [15]:
def get_owner_info(owner_url):
    try:
        response = requests.get(owner_url, headers=get_headers())
        if response.status_code == 200:
            owner_data = response.json()
            return {
                "owner_type": owner_data.get("type", "User"),
                "owner_public_repos": owner_data.get("public_repos", 0),
                "owner_location": owner_data.get("location", None)
            }
    except Exception as e:
        print(f"Erro ao buscar owner: {e}")
    return {
        "owner_type": "User",
        "owner_public_repos": 0,
        "owner_location": None
    }

# 📊 Sistema Avançado de Estatísticas de Repositório

Implementa coleta robusta de métricas detalhadas com paginação completa e filtros temporais:

## Métricas Coletadas:

### 📈 Engajamento da Comunidade
- **Subscribers:** Usuários que recebem notificações (diferente de stars)
- **Contributors:** Desenvolvedores únicos que contribuíram

### 🚀 Atividade de Desenvolvimento
- **Commits Recentes:** Atividade dos últimos 12 meses
- **Issues Fechadas:** Resolução de problemas nos últimos 6 meses
- **Pull Requests:** Total de contribuições externas

## Melhorias Técnicas:

### 🔄 Paginação Inteligente
- Coleta completa até 50 páginas por métrica
- Detecção automática de fim de dados
- Rate limiting entre páginas

### ⏱️ Filtros Temporais
- Issues: Últimos 6 meses para medir atividade recente
- Commits: Últimos 12 meses via API de participação
- Delay estratégico para evitar bloqueios

In [16]:
def get_paginated_count(url, headers, max_pages=50):
    """Função auxiliar para contar itens com paginação completa"""
    total_count = 0
    page = 1
    
    while page <= max_pages:
        try:
            paginated_url = f"{url}?page={page}&per_page=100"
            response = requests.get(paginated_url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                if not data or len(data) == 0:
                    break
                total_count += len(data)
                page += 1
                time.sleep(0.5)  # Delay menor entre páginas
            else:
                break
                
        except Exception as e:
            print(f"Erro na paginação: {e}")
            break
            
    return total_count

def get_repo_stats(owner, repo_name):
    stats = {
        "subscribers_count": 0,
        "last_year_commits": 0,
        "contributors": 0,
        "closed_issues": 0,
        "pull_requests": 0
    }
    
    headers = get_headers()
    
    try:
        # CORREÇÃO: Subscribers com paginação completa
        subscribers_url = f"https://api.github.com/repos/{owner}/{repo_name}/subscribers"
        stats["subscribers_count"] = get_paginated_count(subscribers_url, headers)
        
        # Commits do último ano
        participation = requests.get(
            f"https://api.github.com/repos/{owner}/{repo_name}/stats/participation",
            headers=headers
        )
        if participation.status_code == 200:
            participation_data = participation.json()
            if participation_data and "all" in participation_data:
                stats["last_year_commits"] = sum(participation_data["all"][-52:])

        # CORREÇÃO: Contributors com paginação completa
        contributors_url = f"https://api.github.com/repos/{owner}/{repo_name}/contributors"
        stats["contributors"] = get_paginated_count(contributors_url, headers)

        # CORREÇÃO: Issues fechadas nos últimos 6 meses com paginação
        since_date = (datetime.now() - timedelta(days=180)).isoformat()
        closed_issues_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues"
        
        # Para issues fechadas, precisamos usar parâmetros específicos
        page = 1
        closed_count = 0
        while page <= 20:  # Limite razoável para issues
            try:
                url = f"{closed_issues_url}?state=closed&since={since_date}&page={page}&per_page=100"
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    issues = response.json()
                    if not issues:
                        break
                    closed_count += len(issues)
                    page += 1
                    time.sleep(0.5)
                else:
                    break
            except:
                break
        stats["closed_issues"] = closed_count

        # CORREÇÃO: Pull Requests com paginação completa
        prs_url = f"https://api.github.com/repos/{owner}/{repo_name}/pulls"
        stats["pull_requests"] = get_paginated_count(prs_url, headers)

    except Exception as e:
        print(f"Erro ao buscar stats para {owner}/{repo_name}: {e}")
    
    return stats

# 💾 Sistema de Backup e Salvamento Incremental

**NOVA FUNCIONALIDADE CRÍTICA:** Sistema robusto de persistência de dados que previne perda de informações durante coletas longas.

## Características Principais:

### 🔒 Proteção Contra Perda de Dados
- **Backup Automático:** Arquivos existentes são preservados
- **Salvamento por Linguagem:** Dados salvos após cada linguagem processada
- **Modo Append:** Adiciona dados sem sobrescrever

### 📁 Estrutura de Arquivos
- **Arquivo Principal:** `github_repos_completos.csv`
- **Backups Temporais:** `backup_YYYYMMDD_HHMMSS_arquivo.csv`
- **Backups por Linguagem:** `backup_python_repos.csv`, `backup_javascript_repos.csv`

### 🔄 Recuperação Inteligente
- Detecção de coletas parciais interrompidas
- Continuação automática de onde parou
- Validação de integridade dos dados

**Casos de Uso:**
- ✅ Coleta interrompida por falha de rede
- ✅ Rate limit atingido em linguagens específicas
- ✅ Interrupção manual do usuário
- ✅ Falhas de sistema durante processamento

In [17]:
def save_data_incremental(df, language, is_first_language=False):
    """Salva dados incrementalmente após cada linguagem"""
    try:
        if is_first_language and os.path.exists(OUTPUT_FILE):
            # Backup do arquivo existente
            backup_name = f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{OUTPUT_FILE}"
            os.rename(OUTPUT_FILE, backup_name)
            print(f"Backup criado: {backup_name}")
        
        # Salva ou adiciona ao arquivo
        if not os.path.exists(OUTPUT_FILE) or is_first_language:
            df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8', mode='w')
            print(f"Arquivo criado: {OUTPUT_FILE}")
        else:
            df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8', mode='a', header=False)
            print(f"Dados adicionados ao arquivo: {OUTPUT_FILE}")
            
        # Salva backup específico da linguagem
        lang_file = f"backup_{language.lower()}_repos.csv"
        df.to_csv(lang_file, index=False, encoding='utf-8')
        print(f"Backup da linguagem salvo: {lang_file}")
        
    except Exception as e:
        print(f"Erro ao salvar dados: {e}")

def load_existing_data():
    """Carrega dados existentes se houver"""
    if os.path.exists(OUTPUT_FILE):
        try:
            return pd.read_csv(OUTPUT_FILE)
        except:
            return pd.DataFrame()
    return pd.DataFrame()


# 🔍 Motor de Coleta Principal por Linguagem

Função principal otimizada para coleta eficiente e resiliente de dados do GitHub.

## Recursos Avançados:

### 🛡️ Sistema de Retry Inteligente
- **3 Tentativas** por página com backoff exponencial
- **Detecção de Rate Limit:** Pausa automática de 2 minutos
- **Tratamento de Erros 422:** Detecção de páginas inexistentes

### 📊 Monitoramento em Tempo Real
- Progress tracking detalhado por página e repositório
- Estatísticas de sucesso/falha por linguagem
- Estimativa de tempo restante

### ⚡ Otimizações de Performance
- **Delays Estratégicos:** 2s entre repos, 5s entre páginas
- **Rotação de Tokens:** Distribuição de carga
- **Paralelização Limitada:** Evita sobrecarga da API

## Tratamento de Casos Especiais:
- **Rate Limit (403):** Pausa e retry automático
- **Páginas Vazias:** Detecção de fim de resultados
- **Repositórios Privados:** Skip automático
- **Dados Incompletos:** Preenchimento com valores padrão

In [18]:
def fetch_repos_by_language(language, retry_count=3):
    all_repos = []
    
    print(f"\n🔍 Iniciando coleta para {language}...")
    
    for page in range(1, PAGES + 1):
        for attempt in range(retry_count):
            try:
                url = f"https://api.github.com/search/repositories?q=language:{language}&sort=stars&page={page}&per_page={PER_PAGE}"
                response = requests.get(url, headers=get_headers())
                
                if response.status_code == 403:
                    print(f"Rate limit excedido na página {page}, aguardando...")
                    time.sleep(120)  # Espera 2 minutos
                    continue
                    
                if response.status_code == 422:
                    print(f"Página {page} muito alta, parando coleta para {language}")
                    break
                    
                response.raise_for_status()
                
                repos_data = response.json()
                if "items" not in repos_data:
                    print(f"Sem repositórios na página {page}")
                    break
                
                print(f"  📄 Processando página {page}/{PAGES} ({len(repos_data['items'])} repos)")
                
                for i, repo in enumerate(repos_data["items"]):
                    print(f"    📊 Processando repo {i+1}/{len(repos_data['items'])}: {repo['name']}")
                    
                    repo_info = {
                        "name": repo["name"],
                        "owner": repo["owner"]["login"],
                        "stars": repo["stargazers_count"],
                        "forks": repo["forks_count"],
                        "language": repo["language"],
                        "created_at": repo["created_at"],
                        "updated_at": repo["updated_at"],
                        "size_kb": repo["size"],
                        "watchers_count": repo["watchers_count"],
                        "open_issues": repo["open_issues_count"]
                    }
                    
                    # Buscar informações do owner
                    owner_info = get_owner_info(repo["owner"]["url"])
                    repo_info.update(owner_info)
                    
                    # Buscar estatísticas detalhadas (VERSÃO CORRIGIDA)
                    stats = get_repo_stats(repo["owner"]["login"], repo["name"])
                    repo_info.update(stats)
                    
                    all_repos.append(repo_info)
                    
                    time.sleep(2)  # Delay entre repos
                
                break  # Sai do retry loop se deu certo
                
            except Exception as e:
                print(f"Erro na página {page}, tentativa {attempt + 1}: {e}")
                if attempt == retry_count - 1:
                    print(f"Falha definitiva na página {page}")
                else:
                    time.sleep(30)  # Aguarda antes de tentar novamente
                continue
        
        time.sleep(5)  # Delay entre páginas
            
    return pd.DataFrame(all_repos)

# 🚀 Orquestrador Principal de Execução

Sistema principal que coordena toda a operação de coleta com recursos enterprise-level.

## Funcionalidades Principais:

### 📋 Planejamento Inteligente
- **Detecção de Estado:** Identifica coletas parciais existentes
- **Continuação Automática:** Processa apenas linguagens pendentes
- **Estimativas Precisas:** Cálculo de tempo e volume total

### 🔄 Gestão de Interrupções
- **Ctrl+C Graceful:** Salvamento seguro ao interromper
- **Recovery Automático:** Retoma de onde parou
- **Checkpoint System:** Estados intermediários preservados

### 📊 Monitoramento Avançado
- **Estatísticas em Tempo Real:** Progresso por linguagem
- **Métricas de Performance:** Taxa de sucesso, tempo médio
- **Alertas de Problemas:** Rate limits, falhas de conexão

### 💾 Relatórios Finais
- **Distribuição por Linguagem:** Contagem final por tecnologia
- **Métricas de Arquivo:** Tamanho, localização, integridade
- **Estatísticas de Coleta:** Sucessos, falhas, tempo total

In [20]:
def main():
    print("🚀 Iniciando coleta de dados do GitHub...")
    print(f"📋 Linguagens: {', '.join(LANGUAGES)}")
    print(f"📄 Páginas por linguagem: {PAGES}")
    print(f"📊 Repos por página: {PER_PAGE}")
    print(f"🎯 Total estimado: {len(LANGUAGES) * PAGES * PER_PAGE} repositórios")
    
    # Verifica se já existem dados
    existing_data = load_existing_data()
    if not existing_data.empty:
        print(f"📁 Encontrados {len(existing_data)} repositórios existentes")
        processed_languages = existing_data['language'].unique().tolist()
        remaining_languages = [lang for lang in LANGUAGES if lang not in processed_languages]
        if remaining_languages:
            print(f"🔄 Continuando com: {', '.join(remaining_languages)}")
            languages_to_process = remaining_languages
        else:
            print("✅ Todas as linguagens já foram processadas!")
            return
    else:
        languages_to_process = LANGUAGES
    
    total_collected = len(existing_data)
    
    for i, lang in enumerate(languages_to_process):
        print(f"\n{'='*60}")
        print(f"🎯 LINGUAGEM {i+1}/{len(languages_to_process)}: {lang}")
        print(f"{'='*60}")
        
        try:
            df = fetch_repos_by_language(lang)
            
            if not df.empty:
                # Determina se é a primeira linguagem NOVA sendo processada
                is_first_new = (i == 0 and existing_data.empty)
                
                save_data_incremental(df, lang, is_first_new)
                total_collected += len(df)
                
                print(f"✅ {len(df)} repositórios de {lang} coletados!")
                print(f"📊 Total acumulado: {total_collected} repositórios")
            else:
                print(f"⚠️  Nenhum repositório coletado para {lang}")
            
            # Delay entre linguagens
            if i < len(languages_to_process) - 1:
                print(f"⏳ Aguardando 5 minutos antes da próxima linguagem...")
                time.sleep(300)
                
        except KeyboardInterrupt:
            print(f"\n⚠️  Interrompido pelo usuário na linguagem {lang}")
            print(f"📊 Dados salvos até agora: {total_collected} repositórios")
            break
        except Exception as e:
            print(f"❌ Erro crítico na linguagem {lang}: {e}")
            continue
    
    # Carrega dados finais
    final_data = load_existing_data()
    if not final_data.empty:
        print(f"\n🎉 COLETA FINALIZADA!")
        print(f"📊 Total final: {len(final_data)} repositórios")
        print(f"📁 Arquivo: {OUTPUT_FILE}")
        print(f"💾 Tamanho: {os.path.getsize(OUTPUT_FILE) / 1024 / 1024:.2f} MB")
        
        # Estatísticas por linguagem
        print(f"\n📈 Distribuição por linguagem:")
        lang_stats = final_data['language'].value_counts()
        for lang, count in lang_stats.items():
            print(f"  {lang}: {count} repositórios")
    else:
        print("❌ Nenhum dado foi coletado.")

if __name__ == "__main__":
    main()

🚀 Iniciando coleta de dados do GitHub...
📋 Linguagens: Python, JavaScript, Java, C#, C++, TypeScript, Go, Rust, Kotlin, Swift
📄 Páginas por linguagem: 15
📊 Repos por página: 100
🎯 Total estimado: 15000 repositórios

🎯 LINGUAGEM 1/10: Python

🔍 Iniciando coleta para Python...
  📄 Processando página 1/15 (100 repos)
    📊 Processando repo 1/100: free-programming-books
    📊 Processando repo 2/100: public-apis
    📊 Processando repo 3/100: system-design-primer
    📊 Processando repo 4/100: awesome-python
    📊 Processando repo 5/100: Python
    📊 Processando repo 6/100: AutoGPT
    📊 Processando repo 7/100: stable-diffusion-webui
    📊 Processando repo 8/100: transformers
    📊 Processando repo 9/100: youtube-dl
    📊 Processando repo 10/100: HelloGitHub
    📊 Processando repo 11/100: yt-dlp
    📊 Processando repo 12/100: DeepSeek-V3
    📊 Processando repo 13/100: thefuck
    📊 Processando repo 14/100: pytorch
    📊 Processando repo 15/100: fastapi
    📊 Processando repo 16/100: django
  


# 📋 Dicionário Completo do Dataset

## Estrutura Detalhada dos Dados Coletados

### 🏷️ Informações Básicas do Repositório
| Campo | Tipo | Descrição | Exemplo |
|-------|------|-----------|---------|
| `name` | String | Nome único do repositório | `"tensorflow"` |
| `owner` | String | Username do proprietário | `"google"` |
| `language` | String | Linguagem principal detectada | `"Python"` |

### ⭐ Métricas de Popularidade e Engajamento
| Campo | Tipo | Descrição | Exemplo |
|-------|------|-----------|---------|
| `stars` | Integer | Favoritos dos usuários | `175000` |
| `forks` | Integer | Cópias do repositório | `85000` |
| `watchers_count` | Integer | Usuários observando mudanças | `3200` |
| `subscribers_count` | Integer | Inscritos em notificações | `1500` |

### 🚀 Indicadores de Atividade de Desenvolvimento
| Campo | Tipo | Descrição | Período | Exemplo |
|-------|------|-----------|---------|---------|
| `open_issues` | Integer | Issues não resolvidas | Atual | `42` |
| `closed_issues` | Integer | Issues resolvidas | 6 meses | `128` |
| `pull_requests` | Integer | Total de PRs | Histórico | `75` |
| `last_year_commits` | Integer | Commits realizados | 12 meses | `890` |
| `contributors` | Integer | Desenvolvedores únicos | Histórico | `35` |

### 📅 Metadados Temporais
| Campo | Tipo | Descrição | Formato | Exemplo |
|-------|------|-----------|---------|---------|
| `created_at` | DateTime | Data de criação | ISO 8601 UTC | `2015-11-09T23:25:38Z` |
| `updated_at` | DateTime | Última modificação | ISO 8601 UTC | `2024-03-15T08:12:45Z` |
| `size_kb` | Integer | Tamanho do repositório | Kilobytes | `10240` |

### 👥 Perfil do Proprietário
| Campo | Tipo | Descrição | Valores Possíveis | Exemplo |
|-------|------|-----------|-------------------|---------|
| `owner_type` | String | Tipo de conta | `User`, `Organization` | `"Organization"` |
| `owner_public_repos` | Integer | Repositórios públicos | 0 - ∞ | `250` |
| `owner_location` | String | Localização declarada | Texto livre | `"Mountain View, CA"` |

# 🎯 Casos de Uso e Aplicações dos Dados

## Análises Possíveis com o Dataset:

### 📊 Análise de Tendências Tecnológicas
- **Popularidade por Linguagem:** Ranking de stars e forks
- **Crescimento Temporal:** Evolução de projetos ao longo dos anos
- **Ciclo de Vida:** Correlação entre idade e atividade

### 🌍 Geografia do Código Aberto
- **Mapeamento Global:** Distribuição de desenvolvedores por país
- **Hubs de Inovação:** Identificação de centros tecnológicos
- **Colaboração Internacional:** Projetos multi-regionais

### 👥 Dinâmica de Comunidades
- **Padrões de Contribuição:** Relação contributors vs. atividade
- **Gestão de Issues:** Eficiência na resolução de problemas
- **Sustentabilidade:** Projetos com atividade contínua

### 🏢 Análise Organizacional
- **Usuários vs. Organizações:** Diferentes padrões de desenvolvimento
- **Portfólio Analysis:** Organizações com múltiplos projetos populares
- **Estratégias Open Source:** Abordagens empresariais

# 🔗 Repositório e Recursos Adicionais

## 📂 Acesso ao Código Completo
**GitHub Repository:** https://github.com/LucasjsSilva/data-set-repositorios

### Recursos Disponíveis:
- ✅ Código fonte completo e documentado
- ✅ Scripts de análise de dados
- ✅ Datasets de exemplo
- ✅ Jupyter notebooks com visualizações
- ✅ Guias de instalação e configuração

### Como Contribuir:
1. **Fork** do repositório principal
2. **Clone** para desenvolvimento local
3. **Issues** para reportar bugs ou sugerir melhorias
4. **Pull Requests** com novas funcionalidades